## Ejemplos del HMM Like

### Nombre

* baseline_HMMLike_zero_50_0_0_k_10.crfsuite

### Parametros

* l1 = 0
* l2 = 0
* Precisión = 0.8762
* Max_Iter = 50

In [1]:
import os
import pycrfsuite
import sys  
import random
sys.path.insert(0, '../')
from utils import get_corpus, WordsToLetter, extractLabels, extractTokens
from corpus_utils import (oto_glosser, words_report, gloss_to_csv)

In [2]:
model = "baseline_HMMLike_zero_50_0_0_k_10.crfsuite"
model_path = os.path.join("../models/baseline", "tsu_" + model)
print(model_path)
tagger = pycrfsuite.Tagger()
# Cargando modelos preentrenados
tagger.open(model_path)

../models/baseline/tsu_baseline_HMMLike_zero_50_0_0_k_10.crfsuite


In [3]:
# Obteniendo corpus para pruebas
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + \
         get_corpus('corpus_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)

In [4]:
# funciones auxiliares    
def obtener_palabras(frases):
    palabras = []
    for frase in frases:
        chunks = [palabra[0] for palabra in frase[:-1]]
        palabras.append("".join(chunks))
    return palabras

def accuracy_score(y_test, y_pred):
    right, total = 0, 0
    for real, prediction in zip(y_test, y_pred):
        if real == prediction:
            right += 1
    return right / len(y_test)

def feature_functions_maker(sent):
    ''' Reglas que configuran las feature functions para entrenamiento

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''

    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')
             
            # Add encoding for pysrfsuite
            featurelist.append([f.encode('utf-8') for f in features])
    return featurelist

# Los Peores ejemplos

In [6]:
peores_frases = []
for i, example in enumerate(letter_corpus):
    # Obtenemos las palabras de la frase en forma de lista
    palabras = obtener_palabras(corpus[i])
    # Creación de feature functions
    feature_functions = feature_functions_maker(example)    
    # Etiquetas predichas por el modelo
    prediction_tags = tagger.tag(feature_functions)
    # Etiquetas reales para las partes de las palabras
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    # Accuracy score albitrario para las peores
    if accuracy <= 0.8:
        # Glosador con las etiquetas predichas
        predicted_gloss = oto_glosser(palabras, prediction_tags, corpus[i])
        # Estuctura para ser escrita en csv
        peores_frases.append((palabras, predicted_gloss, i, accuracy))

## Caso donde la generación de *BIO Labels* es erronea

Parece ser que el modelo no aprende la estructura válida para generar BIO Labels que a su vez generan la glosa para los morfemas

In [18]:
palabras = obtener_palabras(corpus[559])
example = letter_corpus[559]
# Creación de feature functions
feature_functions = feature_functions_maker(example)    
# Etiquetas predichas por el modelo
prediction_tags = tagger.tag(feature_functions)
print(prediction_tags)

['I-it', 'B-3.cpl', 'I-3.cpl', 'B-ila', 'I-ila', 'I-ila', 'B-stem', 'I-stem', 'B-stem', 'I-stem', 'B-det', 'I-det', 'B-stem', 'I-stem', 'I-stem', 'I-stem']


In [7]:
# Tomamos los 5 peores
words_report(peores_frases, letter_corpus, feature_functions_maker, tagger, 5)

--------------------------------------------------
Ejemplo 1 de 5 | Frase: "nuní" | Accuracy: 0.0
Letra | Predicción | Real | eq?
n | B-stem | B-det.dem | False
u | I-stem | I-det.dem | False
n | I-stem | I-det.dem | False
í | I-stem | I-det.dem | False
--------------------------------------------------
Ejemplo 2 de 5 | Frase: "ibé" | Accuracy: 0.0
Letra | Predicción | Real | eq?
i | I-it | B-3.icp | False
b | B-dual.exc | B-stem | False
é | I-dual.exc | I-stem | False
--------------------------------------------------
Ejemplo 3 de 5 | Frase: "xirembik'íhí" | Accuracy: 0.08333333333333333
Letra | Predicción | Real | eq?
x | B-stem | B-3.prf | False
i | I-stem | I-3.prf | False
r | I-stem | B-stem | False
e | I-stem | I-stem | True
m | B-med | I-stem | False
b | B-3.cpl | I-stem | False
i | I-3.cpl | I-stem | False
k | B-stem | B-3.obj | False
' | I-stem | I-3.obj | False
í | I-stem | I-3.obj | False
h | I-stem | B-pl | False
í | I-stem | I-pl | False
-----------------------------------

In [8]:
gloss_to_csv(peores_frases, corpus, "peores-frases-HMMLike")

# Mejores ejemplos

Como de esos hay muchos tomaremos 3 al azar

In [13]:
mejores_frases = []
for i, example in enumerate(letter_corpus):
    # Obtenemos las palabras de la frase en forma de lista
    palabras = obtener_palabras(corpus[i])
    # Creación de feature functions
    feature_functions = feature_functions_maker(example)    
    # Etiquetas predichas por el modelo
    prediction_tags = tagger.tag(feature_functions)      
    # Etiquetas reales para las partes de las palabras
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    # Accuracy score albitrario para las peores
    if accuracy >= 0.8:
        # Glosador con las etiquetas predichas
        predicted_gloss = oto_glosser(palabras, prediction_tags, corpus[i])
        # Estuctura para ser escrita en csv
        mejores_frases.append((palabras, predicted_gloss, i, accuracy))

In [14]:
# Tomamos los 5 peores
words_report(mejores_frases, letter_corpus, feature_functions_maker, tagger, 5)

--------------------------------------------------
Ejemplo 1 de 5 | Frase: "digehñawi xo pidómábεphí mähank'ι" | Accuracy: 0.8
Letra | Predicción | Real | eq?
d | B-prt | B-prt | True
i | I-prt | I-prt | True
g | B-mod | B-mod | True
e | I-mod | I-mod | True
h | I-mod | I-mod | True
ñ | I-mod | I-mod | True
a | I-mod | I-mod | True
w | B-prag | B-prag | True
i | I-prag | I-prag | True
x | B-stem | B-stem | True
o | I-stem | I-stem | True
p | B-it | B-it | True
i | I-it | I-it | True
d | B-1.cpl | B-1.cpl | True
ó | I-1.cpl | I-1.cpl | True
m | B-ctrf | B-ctrf | True
á | I-ctrf | I-ctrf | True
b | B-stem | B-stem | True
ε | I-stem | I-stem | True
p | I-stem | I-stem | True
h | I-stem | I-stem | True
í | I-stem | I-stem | True
m | B-1.pss | B-stem | False
ä | I-1.pss | I-stem | False
h | B-stem | I-stem | False
a | I-stem | I-stem | True
n | I-stem | I-stem | True
k | B-dem | I-stem | False
' | I-dem | I-stem | False
ι | I-dem | I-stem | False
--------------------------------------------

In [ ]:
# Escribimos resultados en formato csv
gloss_to_csv(mejores_frases, corpus, "mejores-frases-HMMLike")